In [13]:
import os
import re
import sys
import numpy as np
import pandas as pd
import openpyxl
from glob import glob
from datetime import datetime
from tqdm import tqdm

In [20]:
# read data for an excel file and return a list of dataframes, 
# one for each monitor in the file
def read_xlsx(fpath):
    
    # handy conversion function to handle missing values
    def myconv(strval):
        if strval=='None' or strval==None:
            return np.nan
        else:
            return float(strval)

    workbook = openpyxl.load_workbook(fpath)
    sheet = workbook['Sheet1']
    fname = os.path.basename(fpath)

    for row in sheet.values:
        if row[0] == 'Station':
            templist = row[1].split(' - ')
            agency_names = [ele.split(',', 1)[0] for ele in templist[1:]]
            location_names = [templist[0]] + [ele.split(',',1)[1].strip() for ele in templist[1:-1]]
            monitor_names = ['{} - {}'.format(s1, s2) for s1, s2 in zip(location_names, agency_names)]
            n_monitors = len(monitor_names)
        elif row[0] == 'Parameter':
            params_set = set(row[1].split(','))
        elif row[0] == 'From':
            start_stamp = datetime.strptime(row[1], '%d-%m-%Y T%H:%M:%SZ 00:00')
        elif row[0] == 'To':
            end_stamp = datetime.strptime(row[1], '%d-%m-%Y T%H:%M:%SZ 00:00')
            data_index = pd.date_range(start_stamp, end_stamp, freq='15T', name='timestamp_round')
    
    print('Number of monitors:', n_monitors)
    
    names_rows_list = []
    
    toprow = 12
    while True:
        for cell in sheet[toprow]:
            val = cell.value
            if val is not None and val.strip() != '':
                names_rows_list.append((cell.value.strip(), cell.row+2, cell.column))
        if len(names_rows_list) == n_monitors:
            break
        else:
            toprow += (2 + len(data_index))
    
    #print(names_rows_list)
    
    data_list = []
    
    for loc, start_row, start_col in tqdm(names_rows_list):
        #print(loc, start_row, start_col)
        
        data = []
        start_cell = openpyxl.formula.translate.get_column_letter(start_col) + str(start_row)
        end_cell = openpyxl.formula.translate.get_column_letter(start_col+len(params_set)-1) + str(start_row+len(data_index)-1)
        
        for row in sheet[start_cell:end_cell]:
            readings = [myconv(c.value) for c in row]
            data.append(readings)
        
        df = pd.DataFrame(data, index=data_index, columns=['pm25','pm10'])
        df.sort_index(axis=1, inplace=True) # sort the columns
        
        data_list.append((loc, df))
            
    return data_list

In [31]:
#data_list = read_xlsx('comparision20220209195828.xlsx')
data_list = read_xlsx('comparision20220209195950.xlsx')

100%|██████████| 40/40 [00:00<00:00, 532.30it/s]

Number of monitors: 40


In [32]:
len(data_list)

40

In [33]:
data_list[3]

('Aya Nagar  Delhi - IMD',
                        pm10    pm25
 timestamp_round                    
 2022-01-20 00:00:00  318.97  127.20
 2022-01-20 00:15:00  301.95  122.87
 2022-01-20 00:30:00  285.56  124.58
 2022-01-20 00:45:00  280.78  127.10
 2022-01-20 01:00:00  278.16  146.73
 ...                     ...     ...
 2022-01-22 22:45:00   79.81   56.46
 2022-01-22 23:00:00   86.85   58.84
 2022-01-22 23:15:00   85.45   66.61
 2022-01-22 23:30:00   85.19   69.38
 2022-01-22 23:45:00   89.34   61.99
 
 [288 rows x 2 columns])

In [34]:
# append this raw data to existing table
#data_August2021 = pd.read_csv('Test_Monitors_Aug2021.csv', index_col=0, parse_dates=True)
#data_August2021
data_January2022 = pd.read_csv('Test_Monitors_Jan2022.csv', index_col=0, parse_dates=True)
data_January2022

,Indirapuram_UPPCB,Loni_UPPCB,SanjayNagar_UPPCB,Vasundhara_UPPCB,NoidaSector1_UPPCB,NoidaSector62_IMD,NoidaSector116_UPPCB,NoidaSector125_UPPCB,KnowledgeParkIII_UPPCB,KnowledgeParkV_UPPCB
Timestamp,,,,,,,,,,
2022-01-20 00:00:00,264,275,227,316,195,193.74,142,137,137,128
2022-01-20 00:15:00,264,275,227,316,195,188.30,None,137,137,128
2022-01-20 00:30:00,264,275,227,316,195,190.67,142,137,137,128
2022-01-20 00:45:00,195,272,208,228,182,200.52,134,125,175,132
2022-01-20 01:00:00,195,272,208,None,182,190.38,134,125,175,132
...,...,...,...,...,...,...,...,...,...,...
2022-01-22 22:45:00,62,None,79,70,75,59.39,63,60,69,57
2022-01-22 23:00:00,62,None,79,70,75,65.81,63,60,69,57
2022-01-22 23:15:00,62,None,79,70,75,72.08,63,60,69,57


In [35]:
for loc, data in data_list:
    data_January2022[loc] = data.pm25.values
data_January2022

,Indirapuram_UPPCB,Loni_UPPCB,SanjayNagar_UPPCB,Vasundhara_UPPCB,NoidaSector1_UPPCB,NoidaSector62_IMD,NoidaSector116_UPPCB,NoidaSector125_UPPCB,KnowledgeParkIII_UPPCB,KnowledgeParkV_UPPCB,...,Pusa Delhi - DPCC,Pusa Delhi - IMD,R K Puram Delhi - DPCC,Rohini Delhi - DPCC,Shadipur Delhi - CPCB,Sirifort Delhi - CPCB,Sonia Vihar Delhi - DPCC,Sri Aurobindo Marg Delhi - DPCC,Vivek Vihar Delhi - DPCC,Wazirpur Delhi - DPCC
Timestamp,,,,,,,,,,,,,,,,,,,,,
2022-01-20 00:00:00,264,275,227,316,195,193.74,142,137,137,128,...,241.0,226.92,371.0,373.0,204.4,261.0,317.0,260.0,337.0,389.0
2022-01-20 00:15:00,264,275,227,316,195,188.30,None,137,137,128,...,241.0,240.99,371.0,373.0,204.7,261.0,317.0,260.0,337.0,389.0
2022-01-20 00:30:00,264,275,227,316,195,190.67,142,137,137,128,...,241.0,244.28,371.0,373.0,204.7,261.0,317.0,260.0,337.0,389.0
2022-01-20 00:45:00,195,272,208,228,182,200.52,134,125,175,132,...,273.0,246.57,340.0,320.0,204.7,259.0,277.0,260.0,360.0,389.0
2022-01-20 01:00:00,195,272,208,None,182,190.38,134,125,175,132,...,273.0,235.57,340.0,320.0,183.3,240.0,277.0,230.0,360.0,372.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-22 22:45:00,62,None,79,70,75,59.39,63,60,69,57,...,78.0,56.36,89.0,86.0,113.8,84.0,85.0,102.0,NaN,90.0
2022-01-22 23:00:00,62,None,79,70,75,65.81,63,60,69,57,...,78.0,64.76,89.0,86.0,57.5,84.0,85.0,97.0,NaN,90.0
2022-01-22 23:15:00,62,None,79,70,75,72.08,63,60,69,57,...,78.0,68.25,89.0,86.0,56.9,84.0,85.0,97.0,NaN,90.0


In [36]:
#data_August2021.to_csv('Test_Monitors_Aug2021_new.csv')
data_January2022.to_csv('Test_Monitors_Jan2022_new.csv')